# 데이터 정렬하기

In [1]:
import pandas as pd
customerData = pd.read_csv("../dataset/customerdata.csv")

In [2]:
sortKey = ["EMI", "AVGPRICE"]

In [5]:
# customerData.sort_values(by=sortKey, ascending=[True], ignore_index=True) # ignore_index 는 조심해야 함! 버전에 따라 작동 안 할 수 있음
# customerData.sort_values(by=sortKey, ascending=[True]).reset_index(drop=True)

# customerData.sort_values(by=sortKey, 
#                          ascending=[True],
#                          inplace=True)
# inplace : 즉시 변수에 반영해라!
# 요새는 가독성을 위해 inplace는 잘 쓰지 않는 추세

sortedCustomerData = customerData.sort_values(by=sortKey, ascending=[True, False]).reset_index(drop=True)

In [6]:
sortedCustomerData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
0,K10712,3419.462596,1,4,2.825790,Sleeping-dog
1,O13037,3398.216334,1,4,2.994395,Sleeping-dog
2,I14835,3374.091339,1,4,2.749678,Sleeping-dog
3,M16452,3353.537290,1,4,1.940357,Sleeping-dog
4,C18622,3246.000000,1,5,1.891624,Big-Screen-lover
...,...,...,...,...,...,...
145,F17131,3179.708359,3,6,1.100000,Early-bird
146,J16445,3041.848186,3,6,0.800000,Early-bird
147,I12899,3018.895637,3,6,0.900000,Early-bird
148,X13390,2947.543982,3,6,1.100000,Early-bird


# Group By 

In [6]:
groupKey = ["EMI"]

In [9]:
customerData.groupby(by=groupKey)[["AVGPRICE"]].agg( "mean" ) # 무이자할부 건수 별 구매 가격의 평균

,AVGPRICE
EMI,
1,3119.031810
2,3441.429664
3,3559.487188


In [ ]:
# customerData.groupby(by=groupKey)[["보고 싶은 컬럼"]].agg( "집계함수" ) 집계함수: count, min, max, mean, std(표준편차)

In [12]:
customerData.groupby(by=["EMI"])[["PRODUCTAGE"]].agg("count") # 무이자할부 2번 쓴 사람이 젤 많음?

,PRODUCTAGE
EMI,
1,11
2,120
3,19


In [41]:
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

In [42]:
groupKey = ["REGIONID", "PRODUCT"]

In [45]:
groupData = selloutData.groupby(by=groupKey)[["QTY"]].agg(["mean", "count"]).reset_index()
groupData = groupData.rename( columns={ "mean":"QTY_MEAN",
                                       "count":"QTY_COUNT"})
groupData

REGIONID    PRODUCT            QTY          
                              QTY_MEAN QTY_COUNT
0        A00  PRODUCT34     132.445860       157
1        A00  PRODUCT58     105.235669       157
2        A00  PRODUCT59   36086.547771       157
3        A01   PRODUCT1    5716.726115       157
4        A01  PRODUCT12  197052.777070       157
..       ...        ...            ...       ...
789      A70  PRODUCT34    2165.585987       157
790      A70  PRODUCT58      61.496815       157
791      A70  PRODUCT59   69471.617834       157
792      A77   PRODUCT1    2890.745223       157
793      A77  PRODUCT12    3473.324841       157

[794 rows x 4 columns]

### CUSTOMERDATA 물건구매건수별로 PRODUCTAGE의 평균을 구해본다

In [47]:
groupKey = ["DEVICECOUNT"]

In [48]:
customerData.groupby(by=groupKey)[["PRODUCTAGE"]].agg("mean")

,PRODUCTAGE
DEVICECOUNT,
3,2.233376
4,2.628979
5,2.337668
6,1.155057


# Join

In [108]:
joinMaster = pd.read_csv("../dataset/join_region_master.csv", encoding="ms949")
joinMaster.head()

,REGIONID,REGIONNAME
0,A01,한국
1,A02,미국
2,A03,프랑스
3,A04,독일


In [94]:
joinSellout = pd.read_csv("../dataset/join_kopo_product_volume.csv")
joinSellout.head()

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A01,ST0001,201901,3000
1,A01,ST0001,201902,3500
2,A02,ST0002,201901,4000
3,A02,ST0003,201902,7000


In [95]:
joinKey = ["REGIONID"]

In [96]:
joinMasterKor = joinMaster.loc[joinMaster.REGIONNAME=="한국"]
joinMasterKor

,REGIONID,REGIONNAME
0,A01,한국


In [101]:
mergeResult = pd.merge(left = joinSellout,
                       right = joinMaster,
                       on = joinKey,
                       how = "right") # inner, left, right, full outer
mergeResult

,REGIONID,PRODUCT,YEARWEEK,QTY,REGIONNAME
0,A01,ST0001,201901.0,3000.0,한국
1,A01,ST0001,201902.0,3500.0,한국
2,A02,ST0002,201901.0,4000.0,미국
3,A02,ST0003,201902.0,7000.0,미국
4,A03,NaN,NaN,NaN,프랑스
5,A04,NaN,NaN,NaN,독일


In [107]:
exceptRegion = ["미국"] # 그냥 ~ 써 보려고 이렇게 한 것! 부정의 부정 표현은 실무에서 금지!
joinMasterOuter = joinMaster.loc[~joinMaster.REGIONNAME.isin(exceptRegion)]
joinMasterOuter

,REGIONID,REGIONNAME
0,A01,한국
2,A03,프랑스
3,A04,독일


In [106]:
mergeResult = pd.merge(left = joinSellout,
                       right = joinMasterOuter,
                       on = joinKey,
                       how = "outer") # inner, left, right, full outer
mergeResult

,REGIONID,PRODUCT,YEARWEEK,QTY,REGIONNAME
0,A01,ST0001,201901.0,3000.0,한국
1,A01,ST0001,201902.0,3500.0,한국
2,A02,ST0002,201901.0,4000.0,NaN
3,A02,ST0003,201902.0,7000.0,NaN
4,A03,NaN,NaN,NaN,프랑스
5,A04,NaN,NaN,NaN,독일


# 데이터프레임 만들고 JOIN 실습 해 보기!

In [109]:
joinPromotion = pd.DataFrame([
["A01","COSTCO",0.5],
["A02","BESTBUY",0.3]], columns = ["REGIONID","ACCOUNTID","PROMOTION"])

joinPromoSellout = pd.DataFrame([
["A01","COSTCO","202411",1000],
["A01","COSTCO","202412",2000],
["A02","COSTCO","202412",2000],
["A02","BESTBUY","202411",1000]], columns = ["REGIONID","ACCOUNTID","YEARMONTH","BALANCE"])

In [112]:
joinPromoSellout

,REGIONID,ACCOUNTID,YEARMONTH,BALANCE
0,A01,COSTCO,202411,1000
1,A01,COSTCO,202412,2000
2,A02,COSTCO,202412,2000
3,A02,BESTBUY,202411,1000
